<a href="https://colab.research.google.com/github/Ebasurtos/Machine-Learning/blob/main/Proyecto2_Clasificaci%C3%B3n_Grupo8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Participantes: (Colocar el % de participación)

1 Jorge Palacios 35%

2.Eder Basurto 35%

3 Rodolfo Morocho 30%

Proyecto #2 (Clasificación):

El objetivo de este proyecto es clasificar a los pacientes como con COVID-19, utilizando únicamente el sonido de su tos. Para ello, su grupo puede usar bibliotecas para obtener el mejor vector de características que represente el sonido de la tos. El conjunto de datos contiene las señales sonoras de pacientes con y sin COVID-19. El conjunto de datos se crea a partir de muestras recopiladas de COSWARA y Virufy, que son altamente fiables. Hay 1207 toses de personas con resultado negativo y 150 de personas con resultado positivo de COVID-19

Actividades:

Utilice el conjunto de datos "Toses" y aplique los siguientes algoritmos de clasificación: Regresión logística, SVM, Árboles de decisión y KNN.
Implemente (puntuación sobre 20) o utilice bibliotecas (puntuación sobre 15) para clasificar el conjunto de datos utilizando SVM, KNN y Árboles de decisión.
Realice el proceso de entrenamiento utilizando validación cruzada de K-fold y Bootstrap para estimar el error.
En una tabla, presente los valores de Precisión, Recall y Puntuación F1 para cada prueba de hiperparámetro en cada modelo.
Finalmente, concluya qué modelos ofrecen los mejores resultados

In [1]:
!pip install librosa

import pandas as pd
import numpy as np
import librosa as lb
import os
from sklearn.model_selection import KFold, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Configuración de directorios
base_dir = '/content/drive/MyDrive/ML_Data/tos/cleaned_data'  # Asegúrate de que esta ruta es correcta

# --- Extracción de características mejorada ---
def extract_features(file_path):
    try:
        y, sr = lb.load(file_path, sr=None)  # sr=None para mantener la frecuencia original

        # Extracción de características con parámetros consistentes
        mfccs = lb.feature.mfcc(y=y, sr=sr, n_mfcc=20)
        chroma = lb.feature.chroma_stft(y=y, sr=sr)
        mel = lb.feature.melspectrogram(y=y, sr=sr)
        contrast = lb.feature.spectral_contrast(y=y, sr=sr)
        tonnetz = lb.feature.tonnetz(y=lb.effects.harmonic(y), sr=sr)

        # Calculamos estadísticas para cada característica
        features = np.concatenate([
            np.mean(mfccs, axis=1), np.std(mfccs, axis=1),
            np.mean(chroma, axis=1), np.std(chroma, axis=1),
            np.mean(mel, axis=1), np.std(mel, axis=1),
            np.mean(contrast, axis=1), np.std(contrast, axis=1),
            np.mean(tonnetz, axis=1), np.std(tonnetz, axis=1)
        ])
        return features
    except Exception as e:
        print(f"Error procesando archivo {file_path}: {str(e)}")
        return None

# Procesamiento de datos
def load_data(base_dir):
    features = []
    labels = []

    for label, class_name in enumerate(['Negative', 'Positive']):
        class_dir = os.path.join(base_dir, class_name)
        if not os.path.exists(class_dir):
            print(f"Advertencia: Directorio no encontrado - {class_dir}")
            continue

        for filename in os.listdir(class_dir):
            if filename.endswith('.wav'):
                file_path = os.path.join(class_dir, filename)
                extracted_features = extract_features(file_path)
                if extracted_features is not None:
                    features.append(extracted_features)
                    labels.append(label)

    if not features:
        raise ValueError("No se encontraron archivos de audio válidos. Verifique las rutas y extensiones.")

    return np.array(features), np.array(labels)

try:
    X, y = load_data(base_dir)

    # --- Preprocesamiento ---
    # Dividir datos antes de cualquier transformación para evitar data leakage
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y)

    # --- Modelos con pipeline de escalado ---
    models = {
        'Logistic Regression': make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000)),
        'SVM': make_pipeline(StandardScaler(), SVC()),
        'Decision Tree': DecisionTreeClassifier(),
        'KNN': make_pipeline(StandardScaler(), KNeighborsClassifier())
    }

    results = {}

    # --- Validación Cruzada K-Fold ---
    print("Realizando Validación Cruzada K-Fold...")
    n_splits_kfold = 5
    kf = KFold(n_splits=n_splits_kfold, shuffle=True, random_state=42)

    for name, model in models.items():
        metrics = {
            'Accuracy': [],
            'Precision': [],
            'Recall': [],
            'F1-Score': []
        }

        for train_index, val_index in kf.split(X_train):
            X_train_kf, X_val_kf = X_train[train_index], X_train[val_index]
            y_train_kf, y_val_kf = y_train[train_index], y_train[val_index]

            model.fit(X_train_kf, y_train_kf)
            y_pred_kf = model.predict(X_val_kf)

            metrics['Accuracy'].append(accuracy_score(y_val_kf, y_pred_kf))
            metrics['Precision'].append(precision_score(y_val_kf, y_pred_kf))
            metrics['Recall'].append(recall_score(y_val_kf, y_pred_kf))
            metrics['F1-Score'].append(f1_score(y_val_kf, y_pred_kf))

        results[f'{name}_KFold'] = {
            'Accuracy': np.mean(metrics['Accuracy']),
            'Precision': np.mean(metrics['Precision']),
            'Recall': np.mean(metrics['Recall']),
            'F1-Score': np.mean(metrics['F1-Score'])
        }
        print(f"{name} - KFold completado")

    # --- Bootstrap ---
    print("\nRealizando Bootstrap...")
    n_iterations_bootstrap = 100
    bootstrap_scores = {name: {'Accuracy': [], 'Precision': [], 'Recall': [], 'F1-Score': []}
                       for name in models.keys()}

    for i in range(n_iterations_bootstrap):
        X_train_bs, y_train_bs = resample(X_train, y_train, replace=True, random_state=i)

        for name, model in models.items():
            model.fit(X_train_bs, y_train_bs)
            y_pred_bs = model.predict(X_test)

            bootstrap_scores[name]['Accuracy'].append(accuracy_score(y_test, y_pred_bs))
            bootstrap_scores[name]['Precision'].append(precision_score(y_test, y_pred_bs))
            bootstrap_scores[name]['Recall'].append(recall_score(y_test, y_pred_bs))
            bootstrap_scores[name]['F1-Score'].append(f1_score(y_test, y_pred_bs))

        if (i + 1) % 10 == 0:
            print(f"Iteración Bootstrap {i + 1}/{n_iterations_bootstrap} completada")

    for name in models.keys():
        results[f'{name}_Bootstrap'] = {
            'Accuracy': np.mean(bootstrap_scores[name]['Accuracy']),
            'Precision': np.mean(bootstrap_scores[name]['Precision']),
            'Recall': np.mean(bootstrap_scores[name]['Recall']),
            'F1-Score': np.mean(bootstrap_scores[name]['F1-Score'])
        }

    # --- Presentación de Resultados ---
    results_df = pd.DataFrame.from_dict(results, orient='index')
    print("\nResultados de Clasificación:")
    print(results_df.round(4))

    # --- Análisis de Resultados ---
    print("\nAnálisis Final:")

    # Mejor modelo según K-Fold
    kfold_results = results_df[results_df.index.str.endswith('KFold')]
    best_kfold = kfold_results['F1-Score'].idxmax()
    print(f"\nMejor modelo en Validación Cruzada (F1-Score): {best_kfold}")
    print(kfold_results.loc[best_kfold])

    # Mejor modelo según Bootstrap
    bootstrap_results = results_df[results_df.index.str.endswith('Bootstrap')]
    best_bootstrap = bootstrap_results['F1-Score'].idxmax()
    print(f"\nMejor modelo en Bootstrap (F1-Score): {best_bootstrap}")
    print(bootstrap_results.loc[best_bootstrap])

except Exception as e:
    print(f"\nError en la ejecución: {str(e)}")

/usr/local/lib/python3.11/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=800
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/librosa/core/pitch.py:103: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
/usr/local/lib/python3.11/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=928
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=992
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=768
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=1024 is too large for input signal of length=640
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/librosa/core/spectrum.py:266: 

KeyboardInterrupt: 